Veri setinde eksik olan verilerin üstesinden nasıl gelineceğinin yöntemleri.

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("weather_mis_data.csv")
df

,day,temperature,windspeed,event
0,1/1/2017,32.0,6.0,Rain
1,1/4/2017,NaN,9.0,Sunny
2,1/5/2017,28.0,NaN,Snow
3,1/6/2017,NaN,7.0,NaN
4,1/7/2017,32.0,NaN,Rain
5,1/8/2017,NaN,NaN,Sunny
6,1/9/2017,NaN,NaN,NaN
7,1/10/2017,34.0,8.0,Cloudy
8,1/11/2017,40.0,12.0,Sunny


NaN olan verilerin hepsinin doldurmak için 

In [3]:
# 1. Yöntem
new_df= df.fillna(0) # hepsini 0 ile doldurmak
new_df

,day,temperature,windspeed,event
0,1/1/2017,32.0,6.0,Rain
1,1/4/2017,0.0,9.0,Sunny
2,1/5/2017,28.0,0.0,Snow
3,1/6/2017,0.0,7.0,0
4,1/7/2017,32.0,0.0,Rain
5,1/8/2017,0.0,0.0,Sunny
6,1/9/2017,0.0,0.0,0
7,1/10/2017,34.0,8.0,Cloudy
8,1/11/2017,40.0,12.0,Sunny


In [4]:
# hepsini 0 ile doldurmak çok iyi bir yöntem değil.
# fillna metodu bir sözlük alabilr. özelliklerin hangi değerleri alacağı belirlenebilir

new_df= df.fillna(
{
    'temperature':0,
    'windspeed':0,
    'event':'no event',
})
new_df

,day,temperature,windspeed,event
0,1/1/2017,32.0,6.0,Rain
1,1/4/2017,0.0,9.0,Sunny
2,1/5/2017,28.0,0.0,Snow
3,1/6/2017,0.0,7.0,no event
4,1/7/2017,32.0,0.0,Rain
5,1/8/2017,0.0,0.0,Sunny
6,1/9/2017,0.0,0.0,no event
7,1/10/2017,34.0,8.0,Cloudy
8,1/11/2017,40.0,12.0,Sunny


In [8]:
# Ama bu yöntemler sadece boş durmamasını sağlar bu verilerle bazı hesaplamalar yapılacaksa
# Bu hesaplamalar düzgün sonuçlar vermez. 
# Bunun için bir yöntem önceki günün sıcaklığını bir sonraki eksik veriye taşımak olabilir.

new_df= df.fillna(method="ffill") # bu method önceki günün verisini boş veriye taşır.
# ffill -> önceki günü sonraki boşluğa taşır.
# bfill -> sonraki günü önceki boşluğa taşır.
# Bu özellikler kendinden sonraki tüm boşlukları dolduracak şekilde işler.
# Ancak sadece kendinden sonra 1 değer için yada belirli bir sayıda değer için yapılsın isteniyorsa
# limit=1 gibi bir parametre eklenir. Bu sadece sonraki 1 değer için doldurma işlemi yapar.

#new_df= df.fillna(method="ffill", limit=1)

new_df

,day,temperature,windspeed,event
0,1/1/2017,32.0,6.0,Rain
1,1/4/2017,32.0,9.0,Sunny
2,1/5/2017,28.0,9.0,Snow
3,1/6/2017,28.0,7.0,Snow
4,1/7/2017,32.0,7.0,Rain
5,1/8/2017,32.0,7.0,Sunny
6,1/9/2017,32.0,7.0,Sunny
7,1/10/2017,34.0,8.0,Cloudy
8,1/11/2017,40.0,12.0,Sunny


Verilerde eksik olanları hesaplamaya katmamak için bunları çıkartma işlemi de yapılabilir.

In [9]:
new_df= df.dropna() # bu özellikle satırda 1 tane bile eksik veri varsa çıkartır. 
new_df 

,day,temperature,windspeed,event
0,1/1/2017,32.0,6.0,Rain
7,1/10/2017,34.0,8.0,Cloudy
8,1/11/2017,40.0,12.0,Sunny


In [20]:
# sadece değerlerin tamamı boş olan satır çıkartılmak istenirse
df.set_index('day',inplace=True) # tarihlere göre indexlersek bir satırda veriler tamamen eksik oluyor.
new_df= df.dropna(how='all') # 1/9/2017 olan satır çıakrtılır. 
new_df

,temperature,windspeed,event
day,,,
1/1/2017,32.0,6.0,Rain
1/4/2017,NaN,9.0,Sunny
1/5/2017,28.0,NaN,Snow
1/6/2017,NaN,7.0,NaN
1/7/2017,32.0,NaN,Rain
1/8/2017,NaN,NaN,Sunny
1/10/2017,34.0,8.0,Cloudy
1/11/2017,40.0,12.0,Sunny


In [26]:
# belirli bir değere göre o satır için işlem yapılırsa
# Örneğin 1 veri varsa bu satır tutulsun ve çıkartılmasın

# new_df= df.dropna(thresh=1)

# 2 değer varsa bu işlem için; 
# new_df= df.dropna(thresh=2)

# 3 değer varsa bu işlem için; 
new_df= df.dropna(thresh=3)
new_df


,day,temperature,windspeed,event
0,1/1/2017,32.0,6.0,Rain
1,1/4/2017,NaN,9.0,Sunny
2,1/5/2017,28.0,NaN,Snow
4,1/7/2017,32.0,NaN,Rain
7,1/10/2017,34.0,8.0,Cloudy
8,1/11/2017,40.0,12.0,Sunny


Veri setinde bazı eksik tarih aralığı var. 1 den başlayıp 4 ile devam ediyor.
Arada bulunan 2. ve 3. günler eklenmek istenirse

önce veri setini tarihe göre indexlendi.


df.set_index('day',inplace=True)
df

dt= pd.date_range("01-01-2017","01-11-2017") burada bir tarih aralığı oluşturuldu.


idx=pd.DatetimeIndex(dt) # Bu tarih aralığı dataFramede indexlendi.


df.reindex(idx)# ve bu indexlemeyi kendi dataframe içerisinde yeniden yapıldı.(reindex)

ve bu şekilde veri seti güncellendi

(! verileri silmesi normal değil)

**Replace metoduyla bazı değerler tek tek uğraşmadan değiştirilebilir.**

In [6]:
df2= pd.DataFrame({
    'score':['exceptional', 'average','good','poor','average','exceptional'],
    'student': ['rob','maya','tom','roby','Julian','erica']
})
df2

,score,student
0,exceptional,rob
1,average,maya
2,good,tom
3,poor,roby
4,average,Julian
5,exceptional,erica


In [7]:
# 'poor','average','good' gibi derecelendirilmeleri sayıyla göstermek için;
n_df=df2.replace(['poor','average','good','exceptional'],[1,2,3,4])
n_df

,score,student
0,4,rob
1,2,maya
2,3,tom
3,1,roby
4,2,Julian
5,4,erica
